## Data preparation

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
!pip install catboost
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, cross_val_score
from hyperopt import fmin, tpe, hp, anneal, Trials
from lightgbm.sklearn import LGBMRegressor
import warnings
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 64.8MB 59kB/s 


In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/abdjiber/competiton-zindi-flood-prediction-malawi-data/master/Train.csv")
D= pd.read_csv('https://raw.githubusercontent.com/abdjiber/competiton-zindi-flood-prediction-malawi-data/master/sample_data_basee.csv')
data=(data[data.Square_ID.isin(D.Square_ID)]).reset_index()
del data['index']
print(len(data))
#
precip_2019 = []
precip_2015 = []
for col in data.columns:
    if '2019' in col:
        precip_2019.append(col)
    elif 'precip 2014' in col:
          precip_2015.append(col)
    elif 'precip 2015' in col:
          precip_2015.append(col)
#
data_train = data[data.columns.difference(precip_2019)]
#
precip_2019.extend(['X','Y','elevation', 'LC_Type1_mode','Square_ID'])
data_test = data[precip_2019]

#
new_2015_cols = {}
for col, number in zip(precip_2015, range(1, len(precip_2015) + 1)):
    if 'precip' in col:
        new_2015_cols[col] = 'week_' + str(number) + '_precip'
#
new_2019_cols = {}
for col, number in zip(precip_2019, range(1, len(precip_2019) + 1)):
    if 'precip' in col:
        new_2019_cols[col] = 'week_' + str(number) + '_precip'
# Renaming the columns
data_train.rename(columns = new_2015_cols, inplace = True)
data_test.rename(columns = new_2019_cols, inplace = True)
#
#reoder colums (une façon classique)
data_test=data_test[['LC_Type1_mode','Square_ID', 'X', 'Y', 'elevation', 'week_1_precip',
       'week_2_precip', 'week_3_precip', 'week_4_precip', 'week_5_precip',
       'week_6_precip', 'week_7_precip', 'week_8_precip', 'week_9_precip',
       'week_10_precip', 'week_11_precip', 'week_12_precip', 'week_13_precip',
       'week_14_precip', 'week_15_precip', 'week_16_precip', 'week_17_precip']]
Y_train = data_train['target_2015']
X_train = data_train.drop(['target_2015','Square_ID'],axis=1)
X_test=data_test.drop(['Square_ID'],axis=1)

10181


In [0]:
!pip install haversine
southenr_rivers_lakes_coordinates = {  "Lisungwe": [34.74886, -15.58161],
                                     "Likangala": [35.6, -15.416667], "Naperi": [34.92553, -15.83283], "Mudi": [34.88184, -15.85844],
                                     "Ruo":[35.133333, -16.55], "Lake Malombe": [35.25, -14.666667], "Lake Chilwa": [35.7, -15.3]}
df_rivers = pd.DataFrame(southenr_rivers_lakes_coordinates.values(), columns=["Longitude", "Latitude"], index=southenr_rivers_lakes_coordinates.keys())
#
from haversine import haversine
def compute_distance_from_lake_rivers(df, df_rivers):
    df_distance = pd.DataFrame(np.zeros((df.shape[0], df_rivers.shape[0])), columns=df_rivers.index, index=df.index)
    for i in range(df.shape[0]):
        for j, river in enumerate(df_rivers.index):
            df_distance.iloc[i, j] = haversine((df.Y[i], df.X[i]), (df_rivers.Latitude[j], df_rivers.Longitude[j]))
    df_distance = df_distance.add_prefix('dist_from_')
    #df_distance.to_csv('distances_from_rivers_lakes.csv', header=True, index=True)
    return df_distance
df_distance = compute_distance_from_lake_rivers(data_train, df_rivers)
#
train_dist = pd.concat([data_train, df_distance], axis=1)
test_dist = pd.concat([data_test, df_distance], axis=1)
#
Y_train = train_dist['target_2015']
X_train = train_dist.drop(['target_2015','Square_ID'],axis=1)
X_test=test_dist.drop(['Square_ID'],axis=1)


In [0]:
from sklearn.preprocessing import MinMaxScaler
def scale(train, test):
    scaler = MinMaxScaler()
    scaler.fit(train)
    train_scaled = pd.DataFrame(scaler.transform(train), columns=list(train))
    test_scaled = pd.DataFrame(scaler.transform(test), columns=list(test))
    return train_scaled, test_scaled
train_scaled, test_scaled = scale(X_train, X_test)

# Tuning

## Tuning Xgboosting

In [0]:
#TUNING XGBOOST
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from xgboost import XGBRegressor

# Choose hyperparameter domain to search over
space = {
        'max_depth':hp.choice('max_depth', np.arange(10, 25, 1, dtype=int)),
        'n_estimators':hp.choice('n_estimators', np.arange(100, 10000, 100, dtype=int)),
        'colsample_bytree':hp.quniform('colsample_bytree', 0.5, 1.0, 0.1),
        'min_child_weight':hp.choice('min_child_weight', np.arange(250, 350, 10, dtype=int)),
        'subsample':hp.quniform('subsample', 0.7, 0.9, 0.1),
        'eta':hp.quniform('eta', 0.1, 0.3, 0.1),
        'objective':'reg:squarederror',
        'eval_metric': 'rmse',
    }

def score(params):
    model = XGBRegressor(**params)
    model.fit(train_scaled, Y_train, eval_set=[(train_scaled, Y_train), (test_scaled, Y_train )],
              verbose=False, early_stopping_rounds=10)
    Y_pred = model.predict(test_scaled).clip(0, 20)
    score = np.sqrt(mean_squared_error(Y_train, Y_pred))
    print(score)
    return {'loss': score, 'status': STATUS_OK}    
def optimize(trials, space):
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=50)
    return best
trials = Trials()
best_params = optimize(trials, space)
# Return the best parameters
space_eval(space, best_params)

0.1258598504408217
0.1270634829230783
0.1371483536887085
0.14436170989878594
0.1341962780776364
0.11263916410977695
0.1270634829230783
0.12439542853487842
0.11346241209982491
0.12041317426555218
0.12928031214618987
0.11988423104836946
0.11275328327937342
0.13611836614500908
0.12021332894383369
0.1343239363291416
0.12423360474230526
0.11496971046281326
0.12579860483486485
0.130198153470127
0.10620140067118598
0.11872977295733846
0.11370876228487319
0.11370876228487319
0.11593225559396574
0.11824490234572178
0.11872977295733846
0.12015426205156653
0.09396849164460075
0.09396849164460075
0.09396849164460075
0.09396849164460075
0.10680486862229334
0.12738200129037833
0.09396849164460075
0.11350927504074174
0.11744749319637766
0.10536063054097496
0.11302358607348102
0.10869692187117345
0.10788429693514523
0.11068874746922067
0.12299548817878428
0.12601247127583026
0.11193392817485717
0.11087187752117866
0.11046002001601989
0.0925268718745692
0.12055737579371559
0.10979858014039741
100%|████

{'colsample_bytree': 1.0,
 'eta': 0.30000000000000004,
 'eval_metric': 'rmse',
 'max_depth': 22,
 'min_child_weight': 250,
 'n_estimators': 6600,
 'objective': 'reg:squarederror',
 'subsample': 0.9}

## Tuning LGBM

In [0]:
# Tuning LGBM
from sklearn.model_selection import KFold, cross_val_score
from hyperopt import fmin, tpe, hp, anneal, Trials
from lightgbm.sklearn import LGBMRegressor
random_state=42
n_iter=50
num_folds=2
kf = KFold(n_splits=num_folds, random_state=random_state)

def gb_mse_cv(params, random_state=random_state, cv=kf, X=X_train, y=Y_train):
    # the function gets a set of variable parameters in "param"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'learning_rate': params['learning_rate']}
    
    # we use this params to create a new LGBM Regressor
    model = LGBMRegressor(random_state=random_state, **params)
    
    # and then conduct the cross validation with the same folds as before
    score = -cross_val_score(model, X, y, cv=cv, scoring="neg_mean_squared_error", n_jobs=-1).mean()
    return score

# possible values of parameters
space={'n_estimators': hp.quniform('n_estimators', 100, 2000, 1),
       'max_depth' : hp.quniform('max_depth', 2, 20, 1),
       'learning_rate': hp.loguniform('learning_rate', -5, 0)
      }

# trials will contain logging information
trials = Trials()

best=fmin(fn=gb_mse_cv, # function to optimize
          space=space, 
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=n_iter, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.RandomState(random_state) # fixing random state for the reproducibility
         )

# computing the score on the test set
model = LGBMRegressor(random_state=random_state, n_estimators=int(best['n_estimators']),
                      max_depth=int(best['max_depth']),learning_rate=best['learning_rate'])
model.fit(train_scaled,Y_train) 
tpe_test_score=np.sqrt(mean_squared_error(Y_train, model.predict(test_scaled)))

print("Best MSE {:.3f} params {}".format( gb_mse_cv(best), best))

100%|██████████| 50/50 [03:52<00:00,  4.64s/it, best loss: 0.04005532600253929]
Best MSE 0.040 params {'learning_rate': 0.03944799621468806, 'max_depth': 4.0, 'n_estimators': 1599.0}


## Tuning RandomForest

In [0]:
#tuning randomforest with randomcv
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 100)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3,
                               verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(train_scaled, Y_train)
rf_random.best_params_
model_rf=rf_random.best_estimator_

{'n_estimators': 1251,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 10,
 'bootstrap': True}

In [0]:
def score_rmse(model_rf, test_scaled, Y_train):
    pred_rf = model.predict(test_scaled)
    RMSE= np.sqrt(mean_squared_error(Y_train, pred_rf))
    return RMSE
score_rmse(model_rf, test_scaled, Y_train)

0.15280414344694745

## Tuning catboosting

In [0]:
#tuning catboost with gridseachcv
from sklearn.model_selection import GridSearchCV
model = CatBoostRegressor()
parameters = {'depth'         : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [30, 50, 100]
             }
cat_grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
cat_grid.fit(train_scaled, Y_train) 
print(cat_grid.best_params_)
model_cat=cat_grid.best_estimator_
def score_rmse(model_cat, test_scaled, Y_train):
    pred_cat = model_cat.predict(test_scaled)
    RMSE= np.sqrt(mean_squared_error(Y_train, pred_cat))
    return RMSE
score_rmse(model_cat, test_scaled, Y_train)

0:	learn: 0.2546896	total: 61.5ms	remaining: 6.09s
1:	learn: 0.2468483	total: 70.2ms	remaining: 3.44s
2:	learn: 0.2403268	total: 78.4ms	remaining: 2.53s
3:	learn: 0.2340498	total: 86.5ms	remaining: 2.07s
4:	learn: 0.2281685	total: 94.5ms	remaining: 1.8s
5:	learn: 0.2224874	total: 103ms	remaining: 1.61s
6:	learn: 0.2172266	total: 111ms	remaining: 1.47s
7:	learn: 0.2120470	total: 120ms	remaining: 1.38s
8:	learn: 0.2073710	total: 128ms	remaining: 1.29s
9:	learn: 0.2028880	total: 136ms	remaining: 1.23s
10:	learn: 0.1989308	total: 144ms	remaining: 1.17s
11:	learn: 0.1945955	total: 152ms	remaining: 1.12s
12:	learn: 0.1908013	total: 161ms	remaining: 1.08s
13:	learn: 0.1873211	total: 169ms	remaining: 1.04s
14:	learn: 0.1843081	total: 177ms	remaining: 1s
15:	learn: 0.1811853	total: 185ms	remaining: 972ms
16:	learn: 0.1787510	total: 193ms	remaining: 943ms
17:	learn: 0.1760905	total: 201ms	remaining: 915ms
18:	learn: 0.1738603	total: 210ms	remaining: 894ms
19:	learn: 0.1716365	total: 218ms	remain

0.15099793935510633

## Tuning Gradient Boosting

In [0]:
# Tuning Gradient boosting
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

num_estimators =[100,200,300,400,500,600,700,800,900,1000]
learn_rates = [0.02, 0.05]
max_depths = [1, 2,3,4]
min_samples_leaf = [5,10]
min_samples_split = [5,10]

param_grid = {'n_estimators': num_estimators,
              'learning_rate': learn_rates,
              'max_depth': max_depths,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split}

grid_search = GridSearchCV(GradientBoostingRegressor(loss='huber'),
                           param_grid, cv=3, return_train_score=True)
grid_search.fit(train_scaled, Y_train)
grid_search.best_estimator_

In [0]:

gbm = GradientBoostingRegressor(**params)
preds_oof_gbm, pred_gbm, scores_cv, models = CV_clf(gbm, train_scaled, Y_train, test_scaled) # compt0.125
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019': pred_gbm}) 
submission_df.to_csv('submission_gbm2.csv', index = False)

Fold 0
RMSE 0.08851789328072264
Fold 1
RMSE 0.18381687600664162
Fold 2
RMSE 0.16241380821160126
Fold 3
RMSE 0.1320488212830391
Fold 4
RMSE 0.3406717256099532
Mean RMSE 0.18149382487839155


# CV

In [0]:
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold
RMSE = lambda Y_train, pred: np.sqrt(mean_squared_error(Y_train, pred))

def clip_predictions(preds):
    #preds = preds.where(preds < 1, 1)
    preds= np.where(preds<1,preds,1)
    #preds = preds.where(preds > 0, 0)
    preds= np.where(preds>0,preds,0)
    return preds

def CV_clf(model, train, target, test, n_splits = 5):
    preds_oof = np.zeros(( train.shape[0],), dtype='float')
    pred_test =np.zeros(( test.shape[0],), dtype='float')
    scores_cv = pd.Series([0]*n_splits, dtype=np.float64)
    KF = KFold(n_splits=n_splits,random_state=42)
    models = []
    for i, (idx_train, idx_val) in enumerate(KF.split(train.values, target.values)):
        print('Fold {}'.format(i))
        x_train, y_train = train.values[idx_train], target.values[idx_train]
        x_val, y_val = train.values[idx_val], target.values[idx_val]
        model.fit(x_train,y_train)
        pred_train = model.predict(x_val) 
        preds_oof[idx_val] = pred_train
        pred_test[idx_val] = model.predict(test.values[idx_val])
        score = RMSE(y_val, pred_train)
        models.append(model)
        print('RMSE {}'.format(score))
        scores_cv[i] = score
    print('Mean RMSE', scores_cv.mean())
    pred_train = clip_predictions(preds_oof)
    pred_test = clip_predictions(pred_test)
    return preds_oof, pred_test, scores_cv, models

# Test optimal models

## XGB

In [0]:
#xgboost
#xgb = XGBRegressor(colsample_bytree=1,eta=0.1,eval_metric='rmse',max_depth= 11,min_child_weight=340,
                  #n_estimators=8280,objective='reg:squarederror',subsample= 0.9)
xgb = XGBRegressor(colsample_bytree=1,eta=0.30,eval_metric='rmse',max_depth=22,min_child_weight=250,
n_estimators=6600,objective='reg:squarederror',subsample=0.9)
preds_oof_xgb, pred_test_xgb, scores_cv, models = CV_clf(xgb, train_scaled, Y_train, test_scaled) # compt0.125
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019': pred_test_xgb}) 
submission_df.to_csv('submission_xgb10.csv', index = False)

Fold 0
RMSE 0.09258339726950521
Fold 1
RMSE 0.17042329963460623
Fold 2
RMSE 0.3743220850943004
Fold 3
RMSE 0.11696506571980061
Fold 4
RMSE 0.3435426866854679
Mean RMSE 0.21956730688073609


## LGBM

In [0]:
#LGBM
#lgbm = LGBMRegressor(random_state=42,learning_rate= 0.006761195968100338, max_depth= 2,n_estimators= 681)
lgbm=LGBMRegressor(learning_rate=0.03, max_depth=4, n_estimators=1599)
preds_oof_lgbm, pred_test_lgbm, scores_cv, models = CV_clf(lgbm, train_scaled, Y_train, test_scaled) # compt 0.110
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019': pred_test_lgbm}) 
submission_df.to_csv('submission_lgbm3.csv', index = False)

Fold 0
RMSE 0.07000832026164205
Fold 1
RMSE 0.14696777205145028
Fold 2
RMSE 0.24132529576903441
Fold 3
RMSE 0.07951602582416632
Fold 4
RMSE 0.35224492004858005
Mean RMSE 0.1780124667909746


## randomForest

In [0]:
#radomForest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1251,min_samples_split=10,min_samples_leaf=4,
                            max_features='auto',max_depth=10,bootstrap=True)
preds_oof_rf, pred_test_rf, scores_cv, models = CV_clf(rf, train_scaled, Y_train, test_scaled) # compt 0.15
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019': pred_test_rf}) 
submission_df.to_csv('submission_rf4.csv', index = False)

Fold 0
RMSE 0.07560460585500739
Fold 1
RMSE 0.13690020710175227
Fold 2
RMSE 0.15892076944645925
Fold 3
RMSE 0.10914714507374992
Fold 4
RMSE 0.3159619123282136
Mean RMSE 0.15930692796103646


## catabosting

In [0]:
#cataboost
#cat = CatBoostRegressor(depth=10, iterations=100, learning_rate=0.1)
cat=CatBoostRegressor(depth=6,iterations=100, learning_rate=0.05)
preds_oof_cat, pred_test_cat, scores_cv, models = CV_clf(cat, train_scaled, Y_train, test_scaled) # compt 0.124
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019': pred_test_cat}) 
submission_df.to_csv('submission_cat2.csv', index = False)

Fold 0
0:	learn: 0.2783427	total: 9.94ms	remaining: 984ms
1:	learn: 0.2704840	total: 18ms	remaining: 881ms
2:	learn: 0.2634574	total: 25.5ms	remaining: 825ms
3:	learn: 0.2568690	total: 32.7ms	remaining: 785ms
4:	learn: 0.2502341	total: 39.7ms	remaining: 754ms
5:	learn: 0.2443541	total: 46.8ms	remaining: 733ms
6:	learn: 0.2386854	total: 53.9ms	remaining: 715ms
7:	learn: 0.2335139	total: 61ms	remaining: 702ms
8:	learn: 0.2284709	total: 68.3ms	remaining: 691ms
9:	learn: 0.2243582	total: 75.6ms	remaining: 681ms
10:	learn: 0.2196722	total: 83.2ms	remaining: 673ms
11:	learn: 0.2156945	total: 90.5ms	remaining: 664ms
12:	learn: 0.2120417	total: 97.9ms	remaining: 655ms
13:	learn: 0.2083242	total: 105ms	remaining: 648ms
14:	learn: 0.2046936	total: 113ms	remaining: 640ms
15:	learn: 0.2013149	total: 121ms	remaining: 636ms
16:	learn: 0.1979842	total: 128ms	remaining: 626ms
17:	learn: 0.1949121	total: 140ms	remaining: 639ms
18:	learn: 0.1921130	total: 148ms	remaining: 629ms
19:	learn: 0.1899173	tota

## gradient boosting

In [0]:
# Gradient boosting
gbm =GradientBoostingRegressor(n_estimators=100, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
preds_oof_gbm, pred_test_gbm, scores_cv, models = CV_clf(gbm, train_scaled, Y_train, test_scaled)
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019': pred_test_gbm}) 
submission_df.to_csv('submission_gbm4.csv', index = False)

NameError: ignored

# Stacking

## stack v1

In [0]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_scaled.values)
    rmse= np.sqrt(-cross_val_score(model, train_scaled.values, Y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [0]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [0]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

stacked_averaged_models.fit(train_scaled.values, Y_train)
stacked_pred = stacked_averaged_models.predict(test_scaled.values)
print(rmsle(Y_train, stacked_pred))



Stacking Averaged models score: nan (nan)
0.15563689612514736


In [0]:
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019':stacked_pred}) # compt 0.15
submission_df.to_csv('submission_stack4.csv', index = False)

In [0]:
ensemble1 = stacked_pred*0.10 + pred_test_xgb*0.10 + pred_test_lgbm*0.70 + pred_test_cat*0.10
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019':ensemble}) #
submission_df.to_csv('submission_ensem_stack1.csv', index = False)

In [0]:
ensemble = pred_test_xgb*0.05 + pred_test_lgbm*0.90 + pred_test_cat*0.05
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019':ensemble}) # 0.10
submission_df.to_csv('submission_ensemble6.csv', index = False)

## stack v2

In [0]:
from vecstack import stacking
from sklearn.metrics import mean_absolute_error

#level1
models = [
    XGBRegressor(colsample_bytree=1,eta=0.1,eval_metric='rmse',max_depth= 11,min_child_weight=340,
                  n_estimators=8280,objective='reg:squarederror',subsample= 0.9,tree_method='gpu_hist'),
    CatBoostRegressor(depth=10, iterations=100, learning_rate=0.1),
    LGBMRegressor(random_state=42,learning_rate= 0.006761195968100338, max_depth= 2,n_estimators= 681)

]

#stacking
S_train, S_test = stacking(models,                     # list of models
                           train_scaled, Y_train, test_scaled,   # data
                           regression=True,            # regression task (if you need 
                                                       #     classification - set to False)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and find mean
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=mean_absolute_error, # metric: callable
                           n_folds=4,                  # number of folds
                           shuffle=True,               # shuffle the data
                           random_state=0,             # ensure reproducibility
                           verbose=2)

# level2 model
model = LGBMRegressor(random_state=42,learning_rate= 0.006761195968100338, max_depth= 2,n_estimators= 681)
    
# Fit 2nd level model
model = model.fit(S_train, Y_train)

# Predict
Y_pred_stack = model.predict(S_test)

# Final prediction score
print('Final prediction score: [%.8f]' % np.sqrt(mean_squared_error(Y_train, Y_pred_stack)))

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [XGBRegressor]
    fold  0:  [0.04890965]
    fold  1:  [0.04957701]
    fold  2:  [0.04796417]
    fold  3:  [0.04824200]
    ----
    MEAN:     [0.04867321] + [0.00062479]
    FULL:     [0.04867328]

model  1:     [CatBoostRegressor]
0:	learn: 0.2140611	total: 201ms	remaining: 19.9s
1:	learn: 0.2024577	total: 269ms	remaining: 13.2s
2:	learn: 0.1931459	total: 350ms	remaining: 11.3s
3:	learn: 0.1842378	total: 433ms	remaining: 10.4s
4:	learn: 0.1764487	total: 521ms	remaining: 9.89s
5:	learn: 0.1695594	total: 606ms	remaining: 9.49s
6:	learn: 0.1635009	total: 693ms	remaining: 9.21s
7:	learn: 0.1584856	total: 792ms	remaining: 9.11s
8:	learn: 0.1542803	total: 890ms	remaining: 8.99s
9:	learn: 0.1503996	total: 971ms	remaining: 8.74s
10:	learn: 0.1466470	total: 1.05s	remaining: 8.53s
11:	learn: 0.1436161	total: 1.14s	remaining: 8.37s
12:	learn: 0.1406795	total: 1.23s	rem

54:	learn: 0.1027911	total: 4.77s	remaining: 3.9s
55:	learn: 0.1025182	total: 4.85s	remaining: 3.81s
56:	learn: 0.1020858	total: 4.94s	remaining: 3.72s
57:	learn: 0.1016448	total: 5.03s	remaining: 3.64s
58:	learn: 0.1012874	total: 5.11s	remaining: 3.55s
59:	learn: 0.1010166	total: 5.2s	remaining: 3.46s
60:	learn: 0.1007153	total: 5.28s	remaining: 3.37s
61:	learn: 0.1004973	total: 5.36s	remaining: 3.29s
62:	learn: 0.1002809	total: 5.44s	remaining: 3.2s
63:	learn: 0.1000131	total: 5.53s	remaining: 3.11s
64:	learn: 0.0997393	total: 5.61s	remaining: 3.02s
65:	learn: 0.0993820	total: 5.69s	remaining: 2.93s
66:	learn: 0.0991127	total: 5.78s	remaining: 2.84s
67:	learn: 0.0988018	total: 5.86s	remaining: 2.76s
68:	learn: 0.0985981	total: 5.95s	remaining: 2.67s
69:	learn: 0.0983297	total: 6.03s	remaining: 2.58s
70:	learn: 0.0980405	total: 6.11s	remaining: 2.5s
71:	learn: 0.0977523	total: 6.2s	remaining: 2.41s
72:	learn: 0.0974366	total: 6.28s	remaining: 2.32s
73:	learn: 0.0971406	total: 6.36s	re

15:	learn: 0.1357350	total: 1.34s	remaining: 7.03s
16:	learn: 0.1338324	total: 1.42s	remaining: 6.92s
17:	learn: 0.1319916	total: 1.5s	remaining: 6.86s
18:	learn: 0.1301316	total: 1.59s	remaining: 6.78s
19:	learn: 0.1283900	total: 1.67s	remaining: 6.7s
20:	learn: 0.1268834	total: 1.75s	remaining: 6.6s
21:	learn: 0.1255953	total: 1.84s	remaining: 6.53s
22:	learn: 0.1245785	total: 1.93s	remaining: 6.46s
23:	learn: 0.1236922	total: 2.02s	remaining: 6.39s
24:	learn: 0.1226867	total: 2.1s	remaining: 6.29s
25:	learn: 0.1216072	total: 2.18s	remaining: 6.2s
26:	learn: 0.1207894	total: 2.26s	remaining: 6.12s
27:	learn: 0.1198393	total: 2.35s	remaining: 6.04s
28:	learn: 0.1191333	total: 2.43s	remaining: 5.96s
29:	learn: 0.1183217	total: 2.52s	remaining: 5.88s
30:	learn: 0.1174516	total: 2.6s	remaining: 5.79s
31:	learn: 0.1167449	total: 2.68s	remaining: 5.7s
32:	learn: 0.1161078	total: 2.76s	remaining: 5.61s
33:	learn: 0.1152161	total: 2.85s	remaining: 5.53s
34:	learn: 0.1144134	total: 2.94s	rema

In [0]:
#stacking
clf = LGBMRegressor(random_state=42,learning_rate= 0.006761195968100338, max_depth= 2,n_estimators= 681)
preds_oof_cat, pred_test_stack, scores_cv, models = CV_clf(clf, S_train, Y_train, S_test) # 
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019':pred_test_stack }) # compt 0.120
submission_df.to_csv('submission_stack2.csv', index = False)

Fold 0
RMSE 0.050706237234394526
Fold 1
RMSE 0.08183759649852825
Fold 2
RMSE 0.09931241231735576
Fold 3
RMSE 0.05676470406809669
Fold 4
RMSE 0.14132502275839412
Mean RMSE 0.08598919457535388


# Average models

In [0]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [0]:
averaged_models = AveragingModels(models = (xgb, cat, lgbm, gbm))
score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

averaged_models.fit(train_scaled.values, Y_train)
stacked_avg = averaged_models.predict(test_scaled.values)
print(rmsle(Y_train, stacked_avg))

0:	learn: 0.2365199	total: 72.1ms	remaining: 7.14s
1:	learn: 0.2240998	total: 143ms	remaining: 7.01s
2:	learn: 0.2137045	total: 213ms	remaining: 6.88s
3:	learn: 0.2038105	total: 298ms	remaining: 7.16s
4:	learn: 0.1947024	total: 369ms	remaining: 7.02s
5:	learn: 0.1871937	total: 441ms	remaining: 6.9s
6:	learn: 0.1810685	total: 513ms	remaining: 6.81s
7:	learn: 0.1750789	total: 582ms	remaining: 6.69s
8:	learn: 0.1696723	total: 651ms	remaining: 6.58s
9:	learn: 0.1650991	total: 723ms	remaining: 6.51s
10:	learn: 0.1604913	total: 794ms	remaining: 6.42s
11:	learn: 0.1567874	total: 866ms	remaining: 6.35s
12:	learn: 0.1538647	total: 940ms	remaining: 6.29s
13:	learn: 0.1508208	total: 1.01s	remaining: 6.21s
14:	learn: 0.1485492	total: 1.08s	remaining: 6.11s
15:	learn: 0.1458639	total: 1.15s	remaining: 6.04s
16:	learn: 0.1432572	total: 1.22s	remaining: 5.96s
17:	learn: 0.1411467	total: 1.3s	remaining: 5.92s
18:	learn: 0.1394092	total: 1.38s	remaining: 5.89s
19:	learn: 0.1377013	total: 1.45s	remainin

In [0]:
submission_df = pd.DataFrame({'Square_ID': data_test.Square_ID, 'target_2019':stacked_avg}) # compt 0.120
submission_df.to_csv('submission_average1.csv', index = False)